In [ ]:
# Signal decomposition module

In [28]:
import numpy as np
import pandas as pd

# read SSA.xlsx
file_path = "SSA.xlsx"
SSA = pd.read_excel(file_path)

# read HHT.xlsx
file_path = "HHT.xlsx"
HHT = pd.read_excel(file_path)

# read EMD.xlsx
file_path = "EMD.xlsx"
EMD = pd.read_excel(file_path)

# read HHT.xlsx
file_path = "STL.xlsx"
STL = pd.read_excel(file_path)

In [32]:
# print lowest error for Forecast Type Long 
print("SSA Lowest Error for Forecast Type Long")
print(SSA[SSA["Forecast Type"] == "Long"].sort_values("RMSE").head(5))
print("\n")
print("HHT Lowest Error for Forecast Type Long")
print(HHT[HHT["Forecast Type"] == "Long"].sort_values("RMSE").head(1))
print("\n")
print("EMD Lowest Error for Forecast Type Long")
print(EMD[EMD["Forecast Type"] == "Long"].sort_values("RMSE").head(1))
print("\n")
print("STL Lowest Error for Forecast Type Long")
print(STL[STL["Forecast Type"] == "Long"].sort_values("RMSE").head(1))
print("\n")


SSA Lowest Error for Forecast Type Long
     Location    Disease       RMSE        MAE      MASE       sMAPE  \
35   HongKong     Dengue   8.574597   7.443783  1.949562   65.341828   
26    Iquitos     Dengue  11.104029   8.635363  1.983800  110.756219   
20  Ahmedabad     Dengue  11.319451   8.765273  1.686902  103.919517   
38     Mexico  Influenza  43.262462  25.411300  1.676554   68.666078   
44  Australia  Influenza  81.129843  53.055776  3.340549   93.182978   

   Forecast Type  
35          Long  
26          Long  
20          Long  
38          Long  
44          Long  


HHT Lowest Error for Forecast Type Long
    Location Disease      RMSE       MAE     MASE      sMAPE Forecast Type
35  HongKong  Dengue  5.567871  4.351772  1.13975  45.201046          Long


EMD Lowest Error for Forecast Type Long
   Location Disease       RMSE       MAE      MASE       sMAPE Forecast Type
26  Iquitos  Dengue  12.019937  8.396406  1.928904  148.993978          Long


STL Lowest Error for Fo

In [22]:
print("Columns in shortSSA:", shortSSA.columns)
print("Columns in shortHHT:", shortHHT.columns)
print("Columns in shortEMD:", shortEMD.columns)
print("Columns in shortSTL:", shortSTL.columns)


Columns in shortSSA: Index(['Location', 'Disease', 'SSA_RMSE', 'SSA_MAE', 'SSA_MASE', 'SSA_sMAPE',
       'Forecast Type'],
      dtype='object')
Columns in shortHHT: Index(['Location', 'Disease', 'HHT_RMSE', 'HHT_MAE', 'HHT_MASE', 'HHT_sMAPE',
       'Forecast Type'],
      dtype='object')
Columns in shortEMD: Index(['Location', 'Disease', 'EMD_RMSE', 'EMD_MAE', 'EMD_MASE', 'EMD_sMAPE',
       'Forecast Type'],
      dtype='object')
Columns in shortSTL: Index(['Location', 'Disease', 'STL_RMSE', 'STL_MAE', 'STL_MASE', 'STL_sMAPE',
       'Forecast Type'],
      dtype='object')


In [27]:
# rename RMSE MAE MASE sMAPE in SSA as SSA_RMSE SSA_MAE SSA_MASE SSA_sMAPE
SSA = SSA.rename(columns={"RMSE": "SSA_RMSE", "MAE": "SSA_MAE", "MASE": "SSA_MASE", "sMAPE": "SSA_sMAPE"})
# rename RMSE MAE MASE sMAPE in HHT as HHT_RMSE HHT_MAE HHT_MASE HHT_sMAPE
HHT = HHT.rename(columns={"RMSE": "HHT_RMSE", "MAE": "HHT_MAE", "MASE": "HHT_MASE", "sMAPE": "HHT_sMAPE"})
# rename RMSE MAE MASE sMAPE in EMD as EMD_RMSE EMD_MAE EMD_MASE EMD_sMAPE
EMD = EMD.rename(columns={"RMSE": "EMD_RMSE", "MAE": "EMD_MAE", "MASE": "EMD_MASE", "sMAPE": "EMD_sMAPE"})
# rename RMSE MAE MASE sMAPE in STL as STL_RMSE STL_MAE STL_MASE STL_sMAPE
STL = STL.rename(columns={"RMSE": "STL_RMSE", "MAE": "STL_MAE", "MASE": "STL_MASE", "sMAPE": "STL_sMAPE"})

# Rename Forecast Type in SSA as SSA Forecast Type
SSA = SSA.rename(columns={"Forecast Type": "SSA Forecast Type"})
# Rename Forecast Type in HHT as HHT Forecast Type
HHT = HHT.rename(columns={"Forecast Type": "HHT Forecast Type"})
# Rename Forecast Type in EMD as EMD Forecast Type
EMD = EMD.rename(columns={"Forecast Type": "EMD Forecast Type"})
# Rename Forecast Type in STL as STL Forecast Type
STL = STL.rename(columns={"Forecast Type": "STL Forecast Type"})



shortSSA = SSA[SSA["SSA Forecast Type"] == "Medium"]
shortHHT = HHT[HHT["HHT Forecast Type"] == "Medium"]
shortEMD = EMD[EMD["EMD Forecast Type"] == "Medium"]
shortSTL = STL[STL["STL Forecast Type"] == "Medium"]


# Make a df with rows of Location and Disease is the same in each of the 4 files
# This is the common data
common = pd.merge(shortSSA, shortHHT, on=["Location", "Disease"])
common = pd.merge(common, shortEMD, on=["Location", "Disease"])
common = pd.merge(common, shortSTL, on=["Location", "Disease"])

# save the common data to a file
file_path = "Medium.xlsx"
common.to_excel(file_path, index=False)

In [9]:
# save the dataframe to a new excel file
combined.to_excel('all_methods.xlsx', index=False)
